In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/NicoZenith/Transformers.git

%cd Transformers

Mounted at /content/drive
Cloning into 'Transformers'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 72 (delta 36), reused 33 (delta 11), pack-reused 0
Receiving objects: 100% (72/72), 805.11 KiB | 12.20 MiB/s, done.
Resolving deltas: 100% (36/36), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
/content/Transformers


In [2]:
import torch
import torch.nn as nn

import numpy as np

from transformer_layers import Head

from torch.nn import functional as F

In [3]:
with open('beatles.txt', 'r') as file:
    text = file.read()

print(text[:400])
print("length of dataset in characters", len(text))

A day in the life

I read the news today oh boy
About a lucky man who made the grade
And though the news was rather sad
Well I just had to laugh
I saw the photograph.

He blew his mind out in a car
He didn’t notice that the lights had changed
A crowd of people stood and stared
They'd seen his face before
Nobody was really sure
If he was from the House of Lords.

I saw a film today oh boy
The Engli
length of dataset in characters 168899


In [13]:
from torchtext.data.utils import get_tokenizer

# Define a custom tokenizer that handles line breaks
def custom_tokenizer(text):
    basic_english_tokenizer = get_tokenizer('basic_english')
    tokens = []
    for paragraph in text.split('\n'):
        tokens.extend(basic_english_tokenizer(paragraph))
        # Add a token to represent line breaks
        tokens.append('<br>')
    return tokens


# Tokenize the text using the custom tokenizer
word_tokens = custom_tokenizer(text)
print(word_tokens)

# Collect unique tokens from the dataset
vocab = set()
for data_point in word_tokens:
    vocab.add(data_point)
print(vocab)

vocab = list(vocab)
vocab = ['<pad>'] + vocab
vocab_size = len(vocab)
print("Size of vocabulary =", vocab_size)
vocab_dict = {token: index for index, token in enumerate(vocab)}
vocab_dict_reverse = {index: token for index, token in enumerate(vocab)}

# Encoding and decoding functions
encode = lambda sentence: [vocab_dict[word] for word in custom_tokenizer(sentence)]

# Modify the decode function to return the text with line breaks
def decode(l):
    decoded_sentence = ""
    for integer in l:
        token = vocab_dict_reverse[integer]
        if token == '<br>':
            decoded_sentence += '\n'  # Add line break
        else:
            decoded_sentence += token + ' '
    return decoded_sentence.strip()  # Remove trailing space



# Test the encoding and decoding
encoded_text = encode(text)
decoded_text = decode(encoded_text[:100])
print(decoded_text)


n = int(0.9*len(text)) # first 90% will be train, rest val
train_data = encoded_text[:n]
val_data = encoded_text[n:]


['a', 'day', 'in', 'the', 'life', '<br>', '<br>', 'i', 'read', 'the', 'news', 'today', 'oh', 'boy', '<br>', 'about', 'a', 'lucky', 'man', 'who', 'made', 'the', 'grade', '<br>', 'and', 'though', 'the', 'news', 'was', 'rather', 'sad', '<br>', 'well', 'i', 'just', 'had', 'to', 'laugh', '<br>', 'i', 'saw', 'the', 'photograph', '.', '<br>', '<br>', 'he', 'blew', 'his', 'mind', 'out', 'in', 'a', 'car', '<br>', 'he', 'didn’t', 'notice', 'that', 'the', 'lights', 'had', 'changed', '<br>', 'a', 'crowd', 'of', 'people', 'stood', 'and', 'stared', '<br>', 'they', "'", 'd', 'seen', 'his', 'face', 'before', '<br>', 'nobody', 'was', 'really', 'sure', '<br>', 'if', 'he', 'was', 'from', 'the', 'house', 'of', 'lords', '.', '<br>', '<br>', 'i', 'saw', 'a', 'film', 'today', 'oh', 'boy', '<br>', 'the', 'english', 'army', 'had', 'just', 'won', 'the', 'war', '<br>', 'a', 'crowd', 'of', 'people', 'turned', 'away', '<br>', 'but', 'i', 'just', 'had', 'to', 'look', '<br>', 'having', 'read', 'the', 'book', '.', '<

In [14]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TextDataset(Dataset):
    def __init__(self, text, max_len):
        self.inputs = []
        self.labels = []
        for i in range(len(text) - max_len):
            x = text[i:i+max_len]
            y = text[i+1:i+max_len + 1]
            self.inputs.append(x)
            self.labels.append(y)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.labels[idx])


batch_size = 64
max_len = 8
num_layers = 3
dropout = 0.2


train_dataset = TextDataset(train_data, max_len)
val_dataset = TextDataset(val_data, max_len)


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

x,y = next(iter(train_dataloader))

print(x[0])
print(y[0])




tensor([1404, 1404, 1593, 1652, 2196, 2410, 2557, 1494])
tensor([1404, 1593, 1652, 2196, 2410, 2557, 1494, 1340])


In [15]:
class Cortical_Transformer(nn.Module):
    def __init__(self, d_model, dropout, max_len, vocab_size, num_layers, device):
        super().__init__()
        self.device = device
        self.max_len = max_len
        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        self.position_embedding_table = nn.Embedding(max_len, d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

        self.blocks = nn.Sequential(*[Head(d_model, d_model, dropout) for _ in range(num_layers)])
        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def compute_loss(self, logits, targets):
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return loss


    def forward(self, idx, forward_mask=True):
        """
        Takes a context sequence of tokens as input, and generates the next token. Can be used only for prediction (targets = None)
        or for training.
        :idx: context sequence of tokens
        :valid_lens: actual length of the sequence
        :forward_mask: for unidirectional attention
        :targets: batch of targets for the next token
        """
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, d_model)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.device))
        x = tok_emb + pos_emb # (B, T, d_model)
        for blk in self.blocks:
            x = blk(x, valid_lens=None, forward_mask=forward_mask)
        logits = self.lm_head(x)

        return logits

        # if targets is None:
        #     loss = None
        # else:
        #     B, T, C = logits.shape
        #     logits = logits.view(B*T, C)
        #     targets = targets.view(B*T)
        #     loss = F.cross_entropy(logits, targets)

    def generate(self, idx, max_new_tokens, valid_lens=None, forward_mask=True):
        """
        Generate text by taking a context of size max_len and predicting the next token
        by sampling from the probability distribution obtained with the softmax on the whole vocabulary
        :idx: (B, T) array of indices in the current context
        :max_new_tokens: the number of new tokens we want to generate
        :return: generated sequence including the initial context
        """

        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_len:]
            # get the predictions
            logits = self(idx_cond, valid_lens, forward_mask)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim =1) # (B, T+1)
        return idx


In [16]:
d_model = 24
num_epochs = 7
learning_rate = 1e-4
show_every = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Cortical_Transformer(d_model, dropout, max_len, vocab_size, num_layers, device)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

train_losses = []
val_losses = []


for e in range(num_epochs):
    store_train_losses = []
    store_val_losses = []
    for i, batch in enumerate(train_dataloader):
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        model.zero_grad()

        logits = model(inputs)
        loss = model.compute_loss(logits, targets)
        loss.backward()
        optimizer.step()

        store_train_losses.append(loss.item())

    for i, batch in enumerate(val_dataloader):
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        model.zero_grad()
        with torch.no_grad():
            logits = model(inputs)
            loss = model.compute_loss(logits, targets)

        store_val_losses.append(loss.item())

    train_losses.append(np.mean(store_train_losses))
    val_losses.append(np.mean(store_val_losses))

    print("Epoch {}: Train Loss: {:.4f}, Validation Loss: {:.4f}".format(e, train_losses[-1], val_losses[-1]))









0.13797 M parameters


RuntimeError: ignored